## Quick Tour

The following examples show how to get started with the `unstructured` library. See
our [documentation page](https://unstructured-io.github.io/unstructured) for a full description
of the features in the library.

Another way to try out the `unstructured` library is by running a docker container -- compatible with either Intel/AMD or Apple Silicon! Check out the [instructions for using the docker image](https://github.com/Unstructured-IO/unstructured#dizzy-instructions-for-using-the-docker-image).

In [1]:
# Install Requirements
!apt-get -qq install poppler-utils tesseract-ocr
# Upgrade Pillow to latest version
%pip install -q --user --upgrade pillow
# Install Python Packages
%pip install -q unstructured["all-docs"]==0.12.5
# NOTE: you may also upgrade to the latest version with the command below,
#       though a more recent version of unstructured will not have been tested with this notebook
# %pip install -q --upgrade unstructured

Selecting previously unselected package poppler-utils.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.4_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.4) ...
Selecting previously unselected package tesseract-ocr-eng.
Preparing to unpack .../tesseract-ocr-eng_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr-osd.
Preparing to unpack .../tesseract-ocr-osd_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr.
Preparing to unpack .../tesseract-ocr_4.1.1-2.1build1_amd64.deb ...
Unpacking tesseract-ocr (4.1.1-2.1build1) ...
Setting up tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Setting up poppler-utils (22.02.0-2ubuntu0.4) ...
Setting up tess

See our [example docs page](https://github.com/Unstructured-IO/unstructured/tree/main/example-docs) to find example docs used in this tutorial. You can also upload your own files by clicking on “Choose Files” on the left panel then select and upload the file to Colab.

In [2]:
!mkdir -p example-docs
# Install example-10k.html and layout-parser-paper.pdf
!wget  https://raw.githubusercontent.com/Unstructured-IO/unstructured/main/example-docs/example-10k.html -P example-docs
!wget  https://raw.githubusercontent.com/Unstructured-IO/unstructured/main/example-docs/layout-parser-paper-fast.pdf -P example-docs

--2024-05-23 16:50:02--  https://raw.githubusercontent.com/Unstructured-IO/unstructured/main/example-docs/example-10k.html
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2456707 (2.3M) [text/plain]
Saving to: ‘example-docs/example-10k.html’

example-10k.html    100%[===================>]   2.34M  --.-KB/s    in 0.07s   

2024-05-23 16:50:02 (31.4 MB/s) - ‘example-docs/example-10k.html’ saved [2456707/2456707]

--2024-05-23 16:50:03--  https://raw.githubusercontent.com/Unstructured-IO/unstructured/main/example-docs/layout-parser-paper-fast.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... conn

In [3]:
# Install NLTK Data
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

### HTML Parsing

You can parse an HTML document using the following workflow:

In [ ]:
from unstructured.documents.html import HTMLDocument

doc = HTMLDocument.from_file("example-docs/example-10k.html")

# This is how you would use a document from your google Drive
"""
from google.colab import drive
drive.mount('/content/drive/')
doc = HTMLDocument.from_file("drive/MyDrive/your-filename.html")
"""

'\nfrom google.colab import drive\ndrive.mount(\'/content/drive/\')\ndoc = HTMLDocument.from_file("drive/MyDrive/your-filename.html")\n'

The third page of output looks like the following:

In [ ]:
print(doc.pages[2])

SPECIAL NOTE REGARDING FORWARD-LOOKING STATEMENTS

This report contains statements that do not relate to historical or current facts but are “forward-looking” statements. These statements relate to analyses and other information based on forecasts of future results and estimates of amounts not yet determinable. These statements may also relate to future events or trends, our future prospects and proposed new products, services, developments or business strategies, among other things. These statements can generally (although not always) be identified by their use of terms and phrases such as anticipate, appear, believe, could, would, estimate, expect, indicate, intent, may, plan, predict, project, pursue, will continue and other similar terms and phrases, as well as the use of the future tense.

Actual results could differ materially from those expressed or implied in our forward-looking statements. Our future financial condition and results of operations, as well as any forward-looking

In [ ]:
doc.pages[2].elements

You can see that the parser successfully differentiated between titles and narrative text.

### PDF Parsing

There are two strategies availalbe for parsing PDF documents: "fast" and "hi_res." The default strategy is "hi_res"

If your main objective is extracting text from a "clean" PDF, i.e. one that does not include text in images that require OCR), go with the "fast" option.

Otherwise, if your PDF may have images with text to extract, or, you prefer to have better structured Elements that better characterize the text items within the document, go with with the "hi_res" option.

Naturally, "fast" is faster than "hi_res" -- by an order of magnitude!

In [ ]:
from unstructured.partition.pdf import partition_pdf

# elements = partition_pdf("tata.pdf")

elements_fast = partition_pdf("tata.pdf",
    chunking_strategy="by_title",
    strategy="hi_res",
    extract_images_in_pdf=True,                            # mandatory to set as ``True``
    extract_image_block_types=["Image", "Table"],          # optional
    extract_image_block_to_payload=False,                  # optional
    extract_image_block_output_dir="/content/data",
  )

yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

Let's examine the types of elements returned for both the "hi_res" and "fast" strategies:

In [19]:
from collections import Counter

# display(Counter(type(element) for element in elements))
# print("")
# The composition of elements can be different for elements derived with the "fast" strategy
display(Counter(type(element) for element in elements_fast))

Counter({unstructured.documents.elements.CompositeElement: 692})

In [20]:
display(*[(type(element), element.text) for element in elements_fast])

(unstructured.documents.elements.CompositeElement,
 'OWNER’S MANUAL\n\nRevision: Rev 00/SEPT 2021\n\nCUSTOMER ASSISTANCE\n\nIn our constant endeavour to provide assistance and complete service backup, TATA MOTORS has established an all India cus- tomer assistance centre.\n\nIn case you have a query regarding any aspect of your vehicle, our Customer Assistance Centre will be glad to assist you on our Toll Free no. 1800 209 8282')

(unstructured.documents.elements.CompositeElement,
 'ii\n\nYou can also approach nearest TATA MOTORS dealer.\n\nA separate Dealer network address booklet is provided with the Owner’s manual.\n\nTATA MOTORS 24X7 Roadside Assistance Program offers tech- nical help in the event of a breakdown. Call the toll-free Road- side Assistance.\n\nFor additional information, refer to "24X7 Roadside Assis- tance" section in the Owner’s manual.')

(unstructured.documents.elements.CompositeElement,
 "FOREWORD\n\nDear Customer,\n\nWelcome to the TATA MOTORS family.\n\nWe congratulate you on the purchase of your new vehicle and are privileged to have you as our valued customer.\n\nWe urge you to read this Owner's Manual carefully and familiarize yourself with the equipment descriptions and operating instructions before driving.")

(unstructured.documents.elements.CompositeElement,
 'Always carry out prescribed service / maintenance work as well as any required repairs at an authorized TATA MOTORS Dealers or Authorized Service Centre’s (TASCs). Use only genuine parts for continued reliability, safety and performance of your vehicle.\n\nYou are welcome to contact our dealer or Customer Assistance toll free no. (1800 209 8282) in case of any query or support required.\n\nWe wish you a safe and pleasant driving experience.')

(unstructured.documents.elements.CompositeElement,
 'iii\n\nIMPORTANT INFORMATION •\n\nBefore driving, read this Owner’s manual carefully and familiarize yourself with your vehicle. For your own safety and a longer ve-hicle life, follow the instructions, ‘Warnings’ and ‘Notes’ in this manual. Ignoring them could result in damage to the or personal injury to you or others.\n\nThe Owner’s manual and other booklets are important documents and should always be kept in the vehicle. If you sell the vehicle, always pass on the documents to the new owner.')

...

(unstructured.documents.elements.CompositeElement,
 'Customer’s copy\n\n1st Free Service Coupon\n\nValid for 1000 - 2000 kms. OR 2 months, Whichever is earlier\n\nOwner’s Name:\n\nVIN No.:\n\nEngine / Motor No.:\n\nReg. No.:\n\nODO Reading:\n\nDate of Service:\n\nService Dealer’s Stamp & Signature\n\nDealer’s copy\n\n1st Free Service Coupon\n\nValid for 1000 - 2000 kms. OR 2 months, whichever is earlier.\n\nAVAIL ALL SERVICES TO GET BENEFIT OF WARRANTY\n\nThis Coupon entitles for free labour jobs.(Refer maintenance section)\n\nOwner’s Name:\n\nVIN No.:\n\nEngine / Motor No.:')

(unstructured.documents.elements.CompositeElement,
 'Reg. No.:\n\nODO Reading:\n\nDate of Sale:\n\nDate of Service:\n\nService Dealer code:\n\nI hereby certify that the service has been carried out to my entire satisfaction.\n\nCustomer’s Signature\n\nService Dealer’s Stamp & Signature\n\n(Free Service coupons are valid at all TATA Authorized Service Centers in India)\n\nCustomer’s copy\n\n2nd Free Service Coupon\n\nValid for 7000 - 8000 kms. OR 6 months, whichever is earlier\n\nOwner’s Name:\n\nVIN No.:\n\nEngine / Motor No.:\n\nReg. No.:\n\nODO Reading:\n\nDate of Service:')

(unstructured.documents.elements.CompositeElement,
 'Service Dealer’s Stamp & Signature\n\nDealer’s copy\n\n2nd Free Service Coupon\n\nValid for 7000 - 8000 kms. OR 6 months, whichever is earlier.\n\nAVAIL ALL SERVICES TO GET BENEFIT OF WARRANTY\n\nThis Coupon entitles for free labour jobs.(Refer maintenance section)\n\nOwner’s Name:\n\nVIN No.:\n\nEngine / Motor No.:\n\nReg. No.:\n\nODO Reading:\n\nDate of Sale:\n\nDate of Service:\n\nService Dealer code:\n\nI hereby certify that the service has been carried out to my entire satisfaction.\n\nCustomer’s Signature')

(unstructured.documents.elements.CompositeElement,
 'Service Dealer’s Stamp & Signature\n\n(Free Service coupons are valid at all TATA Authorized Service Centers in India)\n\nCustomer’s copy\n\n3rd Free Service Coupon\n\nValid for 14,500 – 15,500 kms. OR 12 months, whichever is earlier\n\nOwner’s Name:\n\nVIN No.:\n\nEngine / Motor No.:\n\nReg. No.:\n\nODO Reading:\n\nDate of Service:\n\nService Dealer’s Stamp & Signature\n\nDealer’s copy\n\n3rd Free Service Coupon\n\nValid for 14,500 – 15,500 kms. OR 12 months, whichever is earlier.')

(unstructured.documents.elements.CompositeElement,
 'AVAIL ALL SERVICES TO GET BENEFIT OF WARRANTY\n\nThis Coupon entitles for free labour jobs.(Refer maintenance section)\n\nOwner’s Name:\n\nVIN No.:\n\nEngine / Motor No.:\n\nReg. No.:\n\nODO Reading:\n\nDate of Sale:\n\nDate of Service:\n\nService Dealer code:\n\nI hereby certify that the service has been carried out to my entire satisfaction.\n\nCustomer’s Signature\n\nService Dealer’s Stamp & Signature\n\n(Free Service coupons are valid at all TATA Authorized Service Centers in India)')

Let's display the type and text of some of the elements in the document:

You can see that the parser also successfully differentiated between titles and narrative text from a PDF file. However, be aware that element classification is improving as the library evolves, tends to be more accurate with the "hi_res" strategy, and may not always correct.

Now we can join the elements and print the extracted texts from the PDF

In [9]:
print("\n\n".join([str(el) for el in elements]))

Dikshant Khandelwal Roll No.:CS22BTECH11017 B.Tech - Compute Science and Engineering Indian Institute Of Technology, Hyderabad

+91-7014393414 cs22btech11017@iith.ac.in dikkpsd@gmail.com Github | linkedin

Education

Degree/Certificate B.Tech. CSE Senior Secondary Secondary

Institute/Board Indian Institute of Technology, Hyderabad Noble Kingdom Public School Noble Kingdom Public School

CGPA/Percentage 9.65 96.2% 96.83%

Year 2022-Present 2022 2020

Achievements

JEE Advanced , Secured AIR 608 • JEE Mains, Secured AIR 1193

2022 2022

Experience

– Teaching Assistant

JUL23-NOV23

∗Worked as a TA for the course Discrete Mathematics

Skills & Interests

– Programming Languages: Python, C/C++, Dart, Kotlin, Rust, Javscript – Frameworks: Flutter, ExpressJS, FastAPI, Tensorflow, Selenium – Interests: Android Developement, Backend Developement, Reinforcement Learning, Web Scraping – Operating Systems: Windows, Linux – Soft Skills: Communication, team-work, problem-solving

Projects

– Stac